In [26]:
import pandas as pd

# Load HR data for baseline and all three seasons
hr_baseline = pd.read_csv('/Users/urmebose/Desktop/People/Nuro/hr.csv')
hr_01 = pd.read_csv('/Users/urmebose/Desktop/People/Nuro/hr_01.csv')
hr_02 = pd.read_csv('/Users/urmebose/Desktop/People/Nuro/hr_02.csv')
hr_03 = pd.read_csv('/Users/urmebose/Desktop/People/Nuro/hr_03.csv')

# Load psychometric test data for all three seasons
psychometric_01 = pd.read_csv('/Users/urmebose/Desktop/People/Nuro/Psychometric_Test_Results_01.csv')
psychometric_02 = pd.read_csv('/Users/urmebose/Desktop/People/Nuro/Psychometric_Test_Results_02.csv')
psychometric_03 = pd.read_csv('/Users/urmebose/Desktop/People/Nuro/Psychometric_Test_Results_03.csv')

# Clean and prepare the psychometric data
psychometric_01.columns = ['Type', 'Test', 'Question', 'Answer', 'Time(s)', 'Question Start Time', 'Question Answer Time']
psychometric_02.columns = ['Type', 'Test', 'Question', 'Answer', 'Time(s)', 'Question Start Time', 'Question Answer Time']
psychometric_03.columns = ['Type', 'Test', 'Question', 'Answer', 'Time(s)', 'Question Start Time', 'Question Answer Time']

# Convert 'datetime' columns to datetime format and ensure they are tz-naive
hr_baseline['datetime'] = pd.to_datetime(hr_baseline['datetime'], errors='coerce').dt.tz_localize(None)
hr_01['datetime'] = pd.to_datetime(hr_01['datetime'], errors='coerce').dt.tz_localize(None)
hr_02['datetime'] = pd.to_datetime(hr_02['datetime'], errors='coerce').dt.tz_localize(None)
hr_03['datetime'] = pd.to_datetime(hr_03['datetime'], errors='coerce').dt.tz_localize(None)
psychometric_01['Question Start Time'] = pd.to_datetime(psychometric_01['Question Start Time'], errors='coerce').dt.tz_localize(None)
psychometric_02['Question Start Time'] = pd.to_datetime(psychometric_02['Question Start Time'], errors='coerce').dt.tz_localize(None)
psychometric_03['Question Start Time'] = pd.to_datetime(psychometric_03['Question Start Time'], errors='coerce').dt.tz_localize(None)
psychometric_01['Question Answer Time'] = pd.to_datetime(psychometric_01['Question Answer Time'], errors='coerce').dt.tz_localize(None)
psychometric_02['Question Answer Time'] = pd.to_datetime(psychometric_02['Question Answer Time'], errors='coerce').dt.tz_localize(None)
psychometric_03['Question Answer Time'] = pd.to_datetime(psychometric_03['Question Answer Time'], errors='coerce').dt.tz_localize(None)

# Remove rows with NaT values in datetime columns
hr_baseline = hr_baseline.dropna(subset=['datetime'])
hr_01 = hr_01.dropna(subset=['datetime'])
hr_02 = hr_02.dropna(subset=['datetime'])
hr_03 = hr_03.dropna(subset=['datetime'])
psychometric_01 = psychometric_01.dropna(subset=['Question Start Time'])
psychometric_02 = psychometric_02.dropna(subset=['Question Start Time'])
psychometric_03 = psychometric_03.dropna(subset=['Question Start Time'])

# Filter for '14 HADS' questions
questions_14_hads_01 = psychometric_01[(psychometric_01['Type'] == 'HADS') & (psychometric_01['Test'].str.contains('14'))].copy()
questions_14_hads_02 = psychometric_02[(psychometric_02['Type'] == 'HADS') & (psychometric_02['Test'].str.contains('14'))].copy()
questions_14_hads_03 = psychometric_03[(psychometric_03['Type'] == 'HADS') & (psychometric_03['Test'].str.contains('14'))].copy()

def get_average_hr_and_times(questions, hr_data):
    # Merge the psychometric data with heart rate data based on the closest timestamp
    questions.loc[:, 'closest_hr'] = questions['Question Start Time'].apply(
        lambda x: hr_data.iloc[(hr_data['datetime'] - x).abs().argsort()[:1]]['heart_rate'].values[0])
    # Calculate the average heart rate
    average_hr = questions['closest_hr'].mean()
    # Get the start and end times of answering
    start_time = questions['Question Start Time'].min().strftime('%H:%M:%S')
    end_time = questions['Question Answer Time'].max().strftime('%H:%M:%S')
    return average_hr, start_time, end_time

# Calculate average heart rate and times for baseline
baseline_avg_hr = hr_baseline['heart_rate'].mean()

# Calculate average heart rate and times for '14 HADS' during test sessions 01, 02, and 03
average_hr_01, start_time_01, end_time_01 = get_average_hr_and_times(questions_14_hads_01, hr_01)
average_hr_02, start_time_02, end_time_02 = get_average_hr_and_times(questions_14_hads_02, hr_02)
average_hr_03, start_time_03, end_time_03 = get_average_hr_and_times(questions_14_hads_03, hr_03)

# Determine if the HR indicates anxiety based on the baseline HR
anxiety_01 = average_hr_01 > baseline_avg_hr
anxiety_02 = average_hr_02 > baseline_avg_hr
anxiety_03 = average_hr_03 > baseline_avg_hr

# Display the results
print(f'Baseline Average HR: {baseline_avg_hr:.2f} BPM')
print(f'Start time for 14 HADS in Test 01: {start_time_01} - End time: {end_time_01}')
print(f'Average HR during 14 HADS in Test 01: {average_hr_01:.2f} BPM - Anxiety: {"Yes" if anxiety_01 else "No"}')
print(f'Start time for 14 HADS in Test 02: {start_time_02} - End time: {end_time_02}')
print(f'Average HR during 14 HADS in Test 02: {average_hr_02:.2f} BPM - Anxiety: {"Yes" if anxiety_02 else "No"}')
print(f'Start time for 14 HADS in Test 03: {start_time_03} - End time: {end_time_03}')
print(f'Average HR during 14 HADS in Test 03: {average_hr_03:.2f} BPM - Anxiety: {"Yes" if anxiety_03 else "No"}')

Baseline Average HR: 63.43 BPM
Start time for 14 HADS in Test 01: 11:32:02 - End time: 11:32:10
Average HR during 14 HADS in Test 01: 65.00 BPM - Anxiety: Yes
Start time for 14 HADS in Test 02: 09:42:22 - End time: 09:42:25
Average HR during 14 HADS in Test 02: 62.00 BPM - Anxiety: No
Start time for 14 HADS in Test 03: 12:49:36 - End time: 12:49:38
Average HR during 14 HADS in Test 03: 85.00 BPM - Anxiety: Yes


In [24]:
import pandas as pd

# Load HR data
hr_baseline = pd.read_csv('/Users/urmebose/Desktop/People/Nuro/hr.csv')
hr_01 = pd.read_csv('/Users/urmebose/Desktop/People/Nuro/hr_01.csv')
hr_02 = pd.read_csv('/Users/urmebose/Desktop/People/Nuro/hr_02.csv')
hr_03 = pd.read_csv('/Users/urmebose/Desktop/People/Nuro/hr_03.csv')

# Load psychometric test
psychometric_01 = pd.read_csv('/Users/urmebose/Desktop/People/Nuro/Psychometric_Test_Results_01.csv')
psychometric_02 = pd.read_csv('/Users/urmebose/Desktop/People/Nuro/Psychometric_Test_Results_02.csv')
psychometric_03 = pd.read_csv('/Users/urmebose/Desktop/People/Nuro/Psychometric_Test_Results_03.csv')

# Clean and prepare the psychometric data
psychometric_01.columns = ['Type', 'Test', 'Question', 'Answer', 'Time(s)', 'Question Start Time', 'Question Answer Time']
psychometric_02.columns = ['Type', 'Test', 'Question', 'Answer', 'Time(s)', 'Question Start Time', 'Question Answer Time']
psychometric_03.columns = ['Type', 'Test', 'Question', 'Answer', 'Time(s)', 'Question Start Time', 'Question Answer Time']

# Convert 'datetime'
hr_baseline['datetime'] = pd.to_datetime(hr_baseline['datetime'], errors='coerce').dt.tz_localize(None)
hr_01['datetime'] = pd.to_datetime(hr_01['datetime'], errors='coerce').dt.tz_localize(None)
hr_02['datetime'] = pd.to_datetime(hr_02['datetime'], errors='coerce').dt.tz_localize(None)
hr_03['datetime'] = pd.to_datetime(hr_03['datetime'], errors='coerce').dt.tz_localize(None)
psychometric_01['Question Start Time'] = pd.to_datetime(psychometric_01['Question Start Time'], errors='coerce').dt.tz_localize(None)
psychometric_02['Question Start Time'] = pd.to_datetime(psychometric_02['Question Start Time'], errors='coerce').dt.tz_localize(None)
psychometric_03['Question Start Time'] = pd.to_datetime(psychometric_03['Question Start Time'], errors='coerce').dt.tz_localize(None)

# Filtering
hr_baseline = hr_baseline.dropna(subset=['datetime'])
hr_01 = hr_01.dropna(subset=['datetime'])
hr_02 = hr_02.dropna(subset=['datetime'])
hr_03 = hr_03.dropna(subset=['datetime'])
psychometric_01 = psychometric_01.dropna(subset=['Question Start Time'])
psychometric_02 = psychometric_02.dropna(subset=['Question Start Time'])
psychometric_03 = psychometric_03.dropna(subset=['Question Start Time'])

# Filter for '14 HADS'
questions_14_hads_01 = psychometric_01[(psychometric_01['Type'] == 'HADS') & (psychometric_01['Test'].str.contains('14'))].copy()
questions_14_hads_02 = psychometric_02[(psychometric_02['Type'] == 'HADS') & (psychometric_02['Test'].str.contains('14'))].copy()
questions_14_hads_03 = psychometric_03[(psychometric_03['Type'] == 'HADS') & (psychometric_03['Test'].str.contains('14'))].copy()

def get_average_hr_and_start_time(questions, hr_data):
    # Merge the psychometric data with heart rate data based on the closest timestamp
    questions.loc[:, 'closest_hr'] = questions['Question Start Time'].apply(
        lambda x: hr_data.iloc[(hr_data['datetime'] - x).abs().argsort()[:1]]['heart_rate'].values[0])
    # Calculate the average heart rate
    average_hr = questions['closest_hr'].mean()
    # Get the start time of answering
    start_time = questions['Question Start Time'].min()
    return average_hr, start_time

# Calculate average hr
baseline_avg_hr = hr_baseline['heart_rate'].mean()

# Calculate average hr for HADS
average_hr_01, start_time_01 = get_average_hr_and_start_time(questions_14_hads_01, hr_01)
average_hr_02, start_time_02 = get_average_hr_and_start_time(questions_14_hads_02, hr_02)
average_hr_03, start_time_03 = get_average_hr_and_start_time(questions_14_hads_03, hr_03)

# Indicates Anxiety
anxiety_01 = average_hr_01 > baseline_avg_hr
anxiety_02 = average_hr_02 > baseline_avg_hr
anxiety_03 = average_hr_03 > baseline_avg_hr

# Display
print(f'Baseline Average HR: {baseline_avg_hr:.2f} BPM')
print(f'Start time for 14 HADS in Test 01: {start_time_01}')
print(f'Average HR during 14 HADS in Test 01: {average_hr_01:.2f} BPM - Anxiety: {"Yes" if anxiety_01 else "No"}')
print(f'Start time for 14 HADS in Test 02: {start_time_02}')
print(f'Average HR during 14 HADS in Test 02: {average_hr_02:.2f} BPM - Anxiety: {"Yes" if anxiety_02 else "No"}')
print(f'Start time for 14 HADS in Test 03: {start_time_03}')
print(f'Average HR during 14 HADS in Test 03: {average_hr_03:.2f} BPM - Anxiety: {"Yes" if anxiety_03 else "No"}')


Baseline Average HR: 63.43 BPM
Start time for 14 HADS in Test 01: 2024-06-13 11:32:02.833000
Average HR during 14 HADS in Test 01: 65.00 BPM - Anxiety: Yes
Start time for 14 HADS in Test 02: 2024-06-20 09:42:22.376000
Average HR during 14 HADS in Test 02: 62.00 BPM - Anxiety: No
Start time for 14 HADS in Test 03: 2024-06-24 12:49:36.404000
Average HR during 14 HADS in Test 03: 85.00 BPM - Anxiety: Yes


In [28]:
import pandas as pd

# Load HR data for baseline and all three seasons
hr_baseline = pd.read_csv('/Users/urmebose/Desktop/People/Nuro/hr.csv')
hr_01 = pd.read_csv('/Users/urmebose/Desktop/People/Nuro/hr_01.csv')
hr_02 = pd.read_csv('/Users/urmebose/Desktop/People/Nuro/hr_02.csv')
hr_03 = pd.read_csv('/Users/urmebose/Desktop/People/Nuro/hr_03.csv')

# Load psychometric test data for all three seasons
psychometric_01 = pd.read_csv('/Users/urmebose/Desktop/People/Nuro/Psychometric_Test_Results_01.csv')
psychometric_02 = pd.read_csv('/Users/urmebose/Desktop/People/Nuro/Psychometric_Test_Results_02.csv')
psychometric_03 = pd.read_csv('/Users/urmebose/Desktop/People/Nuro/Psychometric_Test_Results_03.csv')

# Clean and prepare the psychometric data
psychometric_01.columns = ['Type', 'Test', 'Question', 'Answer', 'Time(s)', 'Question Start Time', 'Question Answer Time']
psychometric_02.columns = ['Type', 'Test', 'Question', 'Answer', 'Time(s)', 'Question Start Time', 'Question Answer Time']
psychometric_03.columns = ['Type', 'Test', 'Question', 'Answer', 'Time(s)', 'Question Start Time', 'Question Answer Time']

# Convert 'datetime' columns to datetime format and ensure they are tz-naive
hr_baseline['datetime'] = pd.to_datetime(hr_baseline['datetime'], errors='coerce').dt.tz_localize(None)
hr_01['datetime'] = pd.to_datetime(hr_01['datetime'], errors='coerce').dt.tz_localize(None)
hr_02['datetime'] = pd.to_datetime(hr_02['datetime'], errors='coerce').dt.tz_localize(None)
hr_03['datetime'] = pd.to_datetime(hr_03['datetime'], errors='coerce').dt.tz_localize(None)
psychometric_01['Question Start Time'] = pd.to_datetime(psychometric_01['Question Start Time'], errors='coerce').dt.tz_localize(None)
psychometric_02['Question Start Time'] = pd.to_datetime(psychometric_02['Question Start Time'], errors='coerce').dt.tz_localize(None)
psychometric_03['Question Start Time'] = pd.to_datetime(psychometric_03['Question Start Time'], errors='coerce').dt.tz_localize(None)
psychometric_01['Question Answer Time'] = pd.to_datetime(psychometric_01['Question Answer Time'], errors='coerce').dt.tz_localize(None)
psychometric_02['Question Answer Time'] = pd.to_datetime(psychometric_02['Question Answer Time'], errors='coerce').dt.tz_localize(None)
psychometric_03['Question Answer Time'] = pd.to_datetime(psychometric_03['Question Answer Time'], errors='coerce').dt.tz_localize(None)

# Remove rows with NaT values in datetime columns
hr_baseline = hr_baseline.dropna(subset=['datetime'])
hr_01 = hr_01.dropna(subset=['datetime'])
hr_02 = hr_02.dropna(subset=['datetime'])
hr_03 = hr_03.dropna(subset=['datetime'])
psychometric_01 = psychometric_01.dropna(subset=['Question Start Time'])
psychometric_02 = psychometric_02.dropna(subset=['Question Start Time'])
psychometric_03 = psychometric_03.dropna(subset=['Question Start Time'])

# Filter for '14 HADS' questions
questions_14_hads_01 = psychometric_01[(psychometric_01['Type'] == 'HADS') & (psychometric_01['Test'].str.contains('14'))].copy()
questions_14_hads_02 = psychometric_02[(psychometric_02['Type'] == 'HADS') & (psychometric_02['Test'].str.contains('14'))].copy()
questions_14_hads_03 = psychometric_03[(psychometric_03['Type'] == 'HADS') & (psychometric_03['Test'].str.contains('14'))].copy()

def get_average_hr_and_times(questions, hr_data):
    # Merge the psychometric data with heart rate data based on the closest timestamp
    questions.loc[:, 'closest_hr'] = questions['Question Start Time'].apply(
        lambda x: hr_data.iloc[(hr_data['datetime'] - x).abs().argsort()[:1]]['heart_rate'].values[0])
    # Calculate the average heart rate
    average_hr = questions['closest_hr'].mean()
    # Get the start and end times of answering
    start_time = questions['Question Start Time'].min().strftime('%H:%M:%S')
    end_time = questions['Question Answer Time'].max().strftime('%H:%M:%S')
    return average_hr, start_time, end_time

# Calculate average heart rate and times for baseline
baseline_avg_hr = hr_baseline['heart_rate'].mean()

# Calculate average heart rate and times for '14 HADS' during test sessions 01, 02, and 03
average_hr_01, start_time_01, end_time_01 = get_average_hr_and_times(questions_14_hads_01, hr_01)
average_hr_02, start_time_02, end_time_02 = get_average_hr_and_times(questions_14_hads_02, hr_02)
average_hr_03, start_time_03, end_time_03 = get_average_hr_and_times(questions_14_hads_03, hr_03)

# Determine if the HR indicates anxiety based on the baseline HR and general threshold
anxiety_individual_01 = average_hr_01 > baseline_avg_hr
anxiety_general_01 = average_hr_01 > 100
anxiety_individual_02 = average_hr_02 > baseline_avg_hr
anxiety_general_02 = average_hr_02 > 100
anxiety_individual_03 = average_hr_03 > baseline_avg_hr
anxiety_general_03 = average_hr_03 > 100

# Display the results
print(f'Baseline Average HR: {baseline_avg_hr:.2f} BPM')
print(f'Start time for 14 HADS in Test 01: {start_time_01} - End time: {end_time_01}')
print(f'Average HR during 14 HADS in Test 01: {average_hr_01:.2f} BPM - Anxiety (Individual): {"Yes" if anxiety_individual_01 else "No"} - Anxiety (General): {"Yes" if anxiety_general_01 else "No"}')
print(f'Start time for 14 HADS in Test 02: {start_time_02} - End time: {end_time_02}')
print(f'Average HR during 14 HADS in Test 02: {average_hr_02:.2f} BPM - Anxiety (Individual): {"Yes" if anxiety_individual_02 else "No"} - Anxiety (General): {"Yes" if anxiety_general_02 else "No"}')
print(f'Start time for 14 HADS in Test 03: {start_time_03} - End time: {end_time_03}')
print(f'Average HR during 14 HADS in Test 03: {average_hr_03:.2f} BPM - Anxiety (Individual): {"Yes" if anxiety_individual_03 else "No"} - Anxiety (General): {"Yes" if anxiety_general_03 else "No"}')

Baseline Average HR: 63.43 BPM
Start time for 14 HADS in Test 01: 11:32:02 - End time: 11:32:10
Average HR during 14 HADS in Test 01: 65.00 BPM - Anxiety (Individual): Yes - Anxiety (General): No
Start time for 14 HADS in Test 02: 09:42:22 - End time: 09:42:25
Average HR during 14 HADS in Test 02: 62.00 BPM - Anxiety (Individual): No - Anxiety (General): No
Start time for 14 HADS in Test 03: 12:49:36 - End time: 12:49:38
Average HR during 14 HADS in Test 03: 85.00 BPM - Anxiety (Individual): Yes - Anxiety (General): No


In [30]:
import pandas as pd

# Load HR data for baseline and all three seasons
hr_baseline = pd.read_csv('/Users/urmebose/Desktop/People/Nuro/hr.csv')
hr_01 = pd.read_csv('/Users/urmebose/Desktop/People/Nuro/hr_01.csv')
hr_02 = pd.read_csv('/Users/urmebose/Desktop/People/Nuro/hr_02.csv')
hr_03 = pd.read_csv('/Users/urmebose/Desktop/People/Nuro/hr_03.csv')

# Load psychometric test data for all three seasons
psychometric_01 = pd.read_csv('/Users/urmebose/Desktop/People/Nuro/Psychometric_Test_Results_01.csv')
psychometric_02 = pd.read_csv('/Users/urmebose/Desktop/People/Nuro/Psychometric_Test_Results_02.csv')
psychometric_03 = pd.read_csv('/Users/urmebose/Desktop/People/Nuro/Psychometric_Test_Results_03.csv')

# Clean and prepare the psychometric data
psychometric_01.columns = ['Type', 'Test', 'Question', 'Answer', 'Time(s)', 'Question Start Time', 'Question Answer Time']
psychometric_02.columns = ['Type', 'Test', 'Question', 'Answer', 'Time(s)', 'Question Start Time', 'Question Answer Time']
psychometric_03.columns = ['Type', 'Test', 'Question', 'Answer', 'Time(s)', 'Question Start Time', 'Question Answer Time']

# Convert 'datetime' columns to datetime format and ensure they are tz-naive
hr_baseline['datetime'] = pd.to_datetime(hr_baseline['datetime'], errors='coerce').dt.tz_localize(None)
hr_01['datetime'] = pd.to_datetime(hr_01['datetime'], errors='coerce').dt.tz_localize(None)
hr_02['datetime'] = pd.to_datetime(hr_02['datetime'], errors='coerce').dt.tz_localize(None)
hr_03['datetime'] = pd.to_datetime(hr_03['datetime'], errors='coerce').dt.tz_localize(None)
psychometric_01['Question Start Time'] = pd.to_datetime(psychometric_01['Question Start Time'], errors='coerce').dt.tz_localize(None)
psychometric_02['Question Start Time'] = pd.to_datetime(psychometric_02['Question Start Time'], errors='coerce').dt.tz_localize(None)
psychometric_03['Question Start Time'] = pd.to_datetime(psychometric_03['Question Start Time'], errors='coerce').dt.tz_localize(None)
psychometric_01['Question Answer Time'] = pd.to_datetime(psychometric_01['Question Answer Time'], errors='coerce').dt.tz_localize(None)
psychometric_02['Question Answer Time'] = pd.to_datetime(psychometric_02['Question Answer Time'], errors='coerce').dt.tz_localize(None)
psychometric_03['Question Answer Time'] = pd.to_datetime(psychometric_03['Question Answer Time'], errors='coerce').dt.tz_localize(None)

# Remove rows with NaT values in datetime columns
hr_baseline = hr_baseline.dropna(subset=['datetime'])
hr_01 = hr_01.dropna(subset=['datetime'])
hr_02 = hr_02.dropna(subset=['datetime'])
hr_03 = hr_03.dropna(subset=['datetime'])
psychometric_01 = psychometric_01.dropna(subset=['Question Start Time'])
psychometric_02 = psychometric_02.dropna(subset=['Question Start Time'])
psychometric_03 = psychometric_03.dropna(subset=['Question Start Time'])

# Filter for '14 HADS' questions
questions_14_hads_01 = psychometric_01[(psychometric_01['Type'] == 'HADS') & (psychometric_01['Test'].str.contains('14'))].copy()
questions_14_hads_02 = psychometric_02[(psychometric_02['Type'] == 'HADS') & (psychometric_02['Test'].str.contains('14'))].copy()
questions_14_hads_03 = psychometric_03[(psychometric_03['Type'] == 'HADS') & (psychometric_03['Test'].str.contains('14'))].copy()

def get_average_hr_and_times(questions, hr_data):
    # Merge the psychometric data with heart rate data based on the closest timestamp
    questions.loc[:, 'closest_hr'] = questions['Question Start Time'].apply(
        lambda x: hr_data.iloc[(hr_data['datetime'] - x).abs().argsort()[:1]]['heart_rate'].values[0])
    # Calculate the average heart rate
    average_hr = questions['closest_hr'].mean()
    # Get the start and end times of answering
    start_time = questions['Question Start Time'].min().strftime('%H:%M:%S')
    end_time = questions['Question Answer Time'].max().strftime('%H:%M:%S')
    return average_hr, start_time, end_time

# Calculate average heart rate and times for baseline
baseline_avg_hr = hr_baseline['heart_rate'].mean()

# Calculate average heart rate and times for '14 HADS' during test sessions 01, 02, and 03
average_hr_01, start_time_01, end_time_01 = get_average_hr_and_times(questions_14_hads_01, hr_01)
average_hr_02, start_time_02, end_time_02 = get_average_hr_and_times(questions_14_hads_02, hr_02)
average_hr_03, start_time_03, end_time_03 = get_average_hr_and_times(questions_14_hads_03, hr_03)

# Determine if the HR indicates anxiety based on the baseline HR and general threshold
anxiety_individual_01 = average_hr_01 > baseline_avg_hr
anxiety_general_01 = average_hr_01 > 100
anxiety_individual_02 = average_hr_02 > baseline_avg_hr
anxiety_general_02 = average_hr_02 > 100
anxiety_individual_03 = average_hr_03 > baseline_avg_hr
anxiety_general_03 = average_hr_03 > 100

# Display the results in a clear, step-by-step format
print(f"Baseline Average HR: {baseline_avg_hr:.2f} BPM\n")

print(f"Test 01:")
print(f"  Start time for 14 HADS: {start_time_01}")
print(f"  End time for 14 HADS: {end_time_01}")
print(f"  Average HR during 14 HADS: {average_hr_01:.2f} BPM")
print(f"  Anxiety (Individual Threshold): {'Yes' if anxiety_individual_01 else 'No'}")
print(f"  Anxiety (General Threshold >100 BPM): {'Yes' if anxiety_general_01 else 'No'}\n")

print(f"Test 02:")
print(f"  Start time for 14 HADS: {start_time_02}")
print(f"  End time for 14 HADS: {end_time_02}")
print(f"  Average HR during 14 HADS: {average_hr_02:.2f} BPM")
print(f"  Anxiety (Individual Threshold): {'Yes' if anxiety_individual_02 else 'No'}")
print(f"  Anxiety (General Threshold >100 BPM): {'Yes' if anxiety_general_02 else 'No'}\n")

print(f"Test 03:")
print(f"  Start time for 14 HADS: {start_time_03}")
print(f"  End time for 14 HADS: {end_time_03}")
print(f"  Average HR during 14 HADS: {average_hr_03:.2f} BPM")
print(f"  Anxiety (Individual Threshold): {'Yes' if anxiety_individual_03 else 'No'}")
print(f"  Anxiety (General Threshold >100 BPM): {'Yes' if anxiety_general_03 else 'No'}\n")

Baseline Average HR: 63.43 BPM

Test 01:
  Start time for 14 HADS: 11:32:02
  End time for 14 HADS: 11:32:10
  Average HR during 14 HADS: 65.00 BPM
  Anxiety (Individual Threshold): Yes
  Anxiety (General Threshold >100 BPM): No

Test 02:
  Start time for 14 HADS: 09:42:22
  End time for 14 HADS: 09:42:25
  Average HR during 14 HADS: 62.00 BPM
  Anxiety (Individual Threshold): No
  Anxiety (General Threshold >100 BPM): No

Test 03:
  Start time for 14 HADS: 12:49:36
  End time for 14 HADS: 12:49:38
  Average HR during 14 HADS: 85.00 BPM
  Anxiety (Individual Threshold): Yes
  Anxiety (General Threshold >100 BPM): No



In [32]:
import pandas as pd

# Load HR data for baseline and all three seasons
hr_baseline = pd.read_csv('/Users/urmebose/Desktop/People/Nuro/hr.csv')
hr_01 = pd.read_csv('/Users/urmebose/Desktop/People/Nuro/hr_01.csv')
hr_02 = pd.read_csv('/Users/urmebose/Desktop/People/Nuro/hr_02.csv')
hr_03 = pd.read_csv('/Users/urmebose/Desktop/People/Nuro/hr_03.csv')

# Load psychometric test data for all three seasons
psychometric_01 = pd.read_csv('/Users/urmebose/Desktop/People/Nuro/Psychometric_Test_Results_01.csv')
psychometric_02 = pd.read_csv('/Users/urmebose/Desktop/People/Nuro/Psychometric_Test_Results_02.csv')
psychometric_03 = pd.read_csv('/Users/urmebose/Desktop/People/Nuro/Psychometric_Test_Results_03.csv')

# Clean and prepare the psychometric data
psychometric_01.columns = ['Type', 'Test', 'Question', 'Answer', 'Time(s)', 'Question Start Time', 'Question Answer Time']
psychometric_02.columns = ['Type', 'Test', 'Question', 'Answer', 'Time(s)', 'Question Start Time', 'Question Answer Time']
psychometric_03.columns = ['Type', 'Test', 'Question', 'Answer', 'Time(s)', 'Question Start Time', 'Question Answer Time']

# Convert 'datetime' columns to datetime format and ensure they are tz-naive
hr_baseline['datetime'] = pd.to_datetime(hr_baseline['datetime'], errors='coerce').dt.tz_localize(None)
hr_01['datetime'] = pd.to_datetime(hr_01['datetime'], errors='coerce').dt.tz_localize(None)
hr_02['datetime'] = pd.to_datetime(hr_02['datetime'], errors='coerce').dt.tz_localize(None)
hr_03['datetime'] = pd.to_datetime(hr_03['datetime'], errors='coerce').dt.tz_localize(None)
psychometric_01['Question Start Time'] = pd.to_datetime(psychometric_01['Question Start Time'], errors='coerce').dt.tz_localize(None)
psychometric_02['Question Start Time'] = pd.to_datetime(psychometric_02['Question Start Time'], errors='coerce').dt.tz_localize(None)
psychometric_03['Question Start Time'] = pd.to_datetime(psychometric_03['Question Start Time'], errors='coerce').dt.tz_localize(None)
psychometric_01['Question Answer Time'] = pd.to_datetime(psychometric_01['Question Answer Time'], errors='coerce').dt.tz_localize(None)
psychometric_02['Question Answer Time'] = pd.to_datetime(psychometric_02['Question Answer Time'], errors='coerce').dt.tz_localize(None)
psychometric_03['Question Answer Time'] = pd.to_datetime(psychometric_03['Question Answer Time'], errors='coerce').dt.tz_localize(None)

# Remove rows with NaT values in datetime columns
hr_baseline = hr_baseline.dropna(subset=['datetime'])
hr_01 = hr_01.dropna(subset=['datetime'])
hr_02 = hr_02.dropna(subset=['datetime'])
hr_03 = hr_03.dropna(subset=['datetime'])
psychometric_01 = psychometric_01.dropna(subset=['Question Start Time'])
psychometric_02 = psychometric_02.dropna(subset=['Question Start Time'])
psychometric_03 = psychometric_03.dropna(subset=['Question Start Time'])

# Filter for '14 HADS' questions
questions_14_hads_01 = psychometric_01[(psychometric_01['Type'] == 'HADS') & (psychometric_01['Test'].str.contains('14'))].copy()
questions_14_hads_02 = psychometric_02[(psychometric_02['Type'] == 'HADS') & (psychometric_02['Test'].str.contains('14'))].copy()
questions_14_hads_03 = psychometric_03[(psychometric_03['Type'] == 'HADS') & (psychometric_03['Test'].str.contains('14'))].copy()

def get_average_hr_and_times(questions, hr_data):
    # Merge the psychometric data with heart rate data based on the closest timestamp
    questions.loc[:, 'closest_hr'] = questions['Question Start Time'].apply(
        lambda x: hr_data.iloc[(hr_data['datetime'] - x).abs().argsort()[:1]]['heart_rate'].values[0])
    # Calculate the average heart rate
    average_hr = questions['closest_hr'].mean()
    # Get the start and end times of answering
    start_time = questions['Question Start Time'].min().strftime('%H:%M:%S')
    end_time = questions['Question Answer Time'].max().strftime('%H:%M:%S')
    return average_hr, start_time, end_time

# Calculate average heart rate and times for baseline
baseline_avg_hr = hr_baseline['heart_rate'].mean()

# Calculate average heart rate and times for '14 HADS' during test sessions 01, 02, and 03
average_hr_01, start_time_01, end_time_01 = get_average_hr_and_times(questions_14_hads_01, hr_01)
average_hr_02, start_time_02, end_time_02 = get_average_hr_and_times(questions_14_hads_02, hr_02)
average_hr_03, start_time_03, end_time_03 = get_average_hr_and_times(questions_14_hads_03, hr_03)

# Determine if the HR indicates anxiety based on the baseline HR and general threshold
anxiety_individual_01 = average_hr_01 > baseline_avg_hr
anxiety_general_01 = average_hr_01 > 100
anxiety_individual_02 = average_hr_02 > baseline_avg_hr
anxiety_general_02 = average_hr_02 > 100
anxiety_individual_03 = average_hr_03 > baseline_avg_hr
anxiety_general_03 = average_hr_03 > 100

# Create a DataFrame to display the results in a table
results = pd.DataFrame({
    'Test': ['Test 01', 'Test 02', 'Test 03'],
    'Start Time': [start_time_01, start_time_02, start_time_03],
    'End Time': [end_time_01, end_time_02, end_time_03],
    'Average HR (BPM)': [average_hr_01, average_hr_02, average_hr_03],
    'Anxiety (Individual)': ['Yes' if anxiety_individual_01 else 'No', 'Yes' if anxiety_individual_02 else 'No', 'Yes' if anxiety_individual_03 else 'No'],
    'Anxiety (General >100 BPM)': ['Yes' if anxiety_general_01 else 'No', 'Yes' if anxiety_general_02 else 'No', 'Yes' if anxiety_general_03 else 'No']
})

print(f"Baseline Average HR: {baseline_avg_hr:.2f} BPM\n")
print(results)

Baseline Average HR: 63.43 BPM

      Test Start Time  End Time  Average HR (BPM) Anxiety (Individual)  \
0  Test 01   11:32:02  11:32:10              65.0                  Yes   
1  Test 02   09:42:22  09:42:25              62.0                   No   
2  Test 03   12:49:36  12:49:38              85.0                  Yes   

  Anxiety (General >100 BPM)  
0                         No  
1                         No  
2                         No  


In [34]:
import pandas as pd

# Load HR data for baseline and all three seasons
hr_baseline = pd.read_csv('/Users/urmebose/Desktop/People/Nuro/hr.csv')
hr_01 = pd.read_csv('/Users/urmebose/Desktop/People/Nuro/hr_01.csv')
hr_02 = pd.read_csv('/Users/urmebose/Desktop/People/Nuro/hr_02.csv')
hr_03 = pd.read_csv('/Users/urmebose/Desktop/People/Nuro/hr_03.csv')

# Load psychometric test data for all three seasons
psychometric_01 = pd.read_csv('/Users/urmebose/Desktop/People/Nuro/Psychometric_Test_Results_01.csv')
psychometric_02 = pd.read_csv('/Users/urmebose/Desktop/People/Nuro/Psychometric_Test_Results_02.csv')
psychometric_03 = pd.read_csv('/Users/urmebose/Desktop/People/Nuro/Psychometric_Test_Results_03.csv')

# Clean and prepare the psychometric data
psychometric_01.columns = ['Type', 'Test', 'Question', 'Answer', 'Time(s)', 'Question Start Time', 'Question Answer Time']
psychometric_02.columns = ['Type', 'Test', 'Question', 'Answer', 'Time(s)', 'Question Start Time', 'Question Answer Time']
psychometric_03.columns = ['Type', 'Test', 'Question', 'Answer', 'Time(s)', 'Question Start Time', 'Question Answer Time']

# Convert 'datetime' columns to datetime format and ensure they are tz-naive
hr_baseline['datetime'] = pd.to_datetime(hr_baseline['datetime'], errors='coerce').dt.tz_localize(None)
hr_01['datetime'] = pd.to_datetime(hr_01['datetime'], errors='coerce').dt.tz_localize(None)
hr_02['datetime'] = pd.to_datetime(hr_02['datetime'], errors='coerce').dt.tz_localize(None)
hr_03['datetime'] = pd.to_datetime(hr_03['datetime'], errors='coerce').dt.tz_localize(None)
psychometric_01['Question Start Time'] = pd.to_datetime(psychometric_01['Question Start Time'], errors='coerce').dt.tz_localize(None)
psychometric_02['Question Start Time'] = pd.to_datetime(psychometric_02['Question Start Time'], errors='coerce').dt.tz_localize(None)
psychometric_03['Question Start Time'] = pd.to_datetime(psychometric_03['Question Start Time'], errors='coerce').dt.tz_localize(None)
psychometric_01['Question Answer Time'] = pd.to_datetime(psychometric_01['Question Answer Time'], errors='coerce').dt.tz_localize(None)
psychometric_02['Question Answer Time'] = pd.to_datetime(psychometric_02['Question Answer Time'], errors='coerce').dt.tz_localize(None)
psychometric_03['Question Answer Time'] = pd.to_datetime(psychometric_03['Question Answer Time'], errors='coerce').dt.tz_localize(None)

# Remove rows with NaT values in datetime columns
hr_baseline = hr_baseline.dropna(subset=['datetime'])
hr_01 = hr_01.dropna(subset=['datetime'])
hr_02 = hr_02.dropna(subset=['datetime'])
hr_03 = hr_03.dropna(subset=['datetime'])
psychometric_01 = psychometric_01.dropna(subset=['Question Start Time'])
psychometric_02 = psychometric_02.dropna(subset=['Question Start Time'])
psychometric_03 = psychometric_03.dropna(subset=['Question Start Time'])

# Filter for '14 HADS' questions
questions_14_hads_01 = psychometric_01[(psychometric_01['Type'] == 'HADS') & (psychometric_01['Test'].str.contains('14'))].copy()
questions_14_hads_02 = psychometric_02[(psychometric_02['Type'] == 'HADS') & (psychometric_02['Test'].str.contains('14'))].copy()
questions_14_hads_03 = psychometric_03[(psychometric_03['Type'] == 'HADS') & (psychometric_03['Test'].str.contains('14'))].copy()

def get_average_hr_and_times(questions, hr_data):
    # Merge the psychometric data with heart rate data based on the closest timestamp
    questions.loc[:, 'closest_hr'] = questions['Question Start Time'].apply(
        lambda x: hr_data.iloc[(hr_data['datetime'] - x).abs().argsort()[:1]]['heart_rate'].values[0])
    # Calculate the average heart rate
    average_hr = questions['closest_hr'].mean()
    # Get the start and end times of answering
    start_time = questions['Question Start Time'].min().strftime('%H:%M:%S')
    end_time = questions['Question Answer Time'].max().strftime('%H:%M:%S')
    return average_hr, start_time, end_time

# Calculate average heart rate and times for baseline
baseline_avg_hr = hr_baseline['heart_rate'].mean()

# Calculate average heart rate and times for '14 HADS' during test sessions 01, 02, and 03
average_hr_01, start_time_01, end_time_01 = get_average_hr_and_times(questions_14_hads_01, hr_01)
average_hr_02, start_time_02, end_time_02 = get_average_hr_and_times(questions_14_hads_02, hr_02)
average_hr_03, start_time_03, end_time_03 = get_average_hr_and_times(questions_14_hads_03, hr_03)

# Determine if the HR indicates anxiety based on the baseline HR and general threshold
anxiety_individual_01 = average_hr_01 > baseline_avg_hr
anxiety_general_01 = average_hr_01 > 100
anxiety_individual_02 = average_hr_02 > baseline_avg_hr
anxiety_general_02 = average_hr_02 > 100
anxiety_individual_03 = average_hr_03 > baseline_avg_hr
anxiety_general_03 = average_hr_03 > 100

# Create a DataFrame to display the results in a table
results = pd.DataFrame({
    'Test': ['Test 01', 'Test 02', 'Test 03'],
    'Start Time': [start_time_01, start_time_02, start_time_03],
    'End Time': [end_time_01, end_time_02, end_time_03],
    'Average HR (BPM)': [average_hr_01, average_hr_02, average_hr_03],
    'Anxiety (Individual)': ['Yes' if anxiety_individual_01 else 'No', 'Yes' if anxiety_individual_02 else 'No', 'Yes' if anxiety_individual_03 else 'No'],
    'Anxiety (General >100 BPM)': ['Yes' if anxiety_general_01 else 'No', 'Yes' if anxiety_general_02 else 'No', 'Yes' if anxiety_general_03 else 'No']
})

# Display the results more clearly
print(f"Baseline Average HR: {baseline_avg_hr:.2f} BPM\n")

for index, row in results.iterrows():
    print(f"{row['Test']}:")
    print(f"  Start time for 14 HADS: {row['Start Time']}")
    print(f"  End time for 14 HADS: {row['End Time']}")
    print(f"  Average HR during 14 HADS: {row['Average HR (BPM)']:.2f} BPM")
    print(f"  Anxiety (Individual Threshold): {row['Anxiety (Individual)']}")
    print(f"  Anxiety (General Threshold >100 BPM): {row['Anxiety (General >100 BPM)']}\n")


Baseline Average HR: 63.43 BPM

Test 01:
  Start time for 14 HADS: 11:32:02
  End time for 14 HADS: 11:32:10
  Average HR during 14 HADS: 65.00 BPM
  Anxiety (Individual Threshold): Yes
  Anxiety (General Threshold >100 BPM): No

Test 02:
  Start time for 14 HADS: 09:42:22
  End time for 14 HADS: 09:42:25
  Average HR during 14 HADS: 62.00 BPM
  Anxiety (Individual Threshold): No
  Anxiety (General Threshold >100 BPM): No

Test 03:
  Start time for 14 HADS: 12:49:36
  End time for 14 HADS: 12:49:38
  Average HR during 14 HADS: 85.00 BPM
  Anxiety (Individual Threshold): Yes
  Anxiety (General Threshold >100 BPM): No



In [36]:
import pandas as pd

# Load HR data for baseline and all three seasons
hr_baseline = pd.read_csv('/Users/urmebose/Desktop/People/Nuro/hr.csv')
hr_01 = pd.read_csv('/Users/urmebose/Desktop/People/Nuro/hr_01.csv')
hr_02 = pd.read_csv('/Users/urmebose/Desktop/People/Nuro/hr_02.csv')
hr_03 = pd.read_csv('/Users/urmebose/Desktop/People/Nuro/hr_03.csv')

# Load psychometric test data for all three seasons
psychometric_01 = pd.read_csv('/Users/urmebose/Desktop/People/Nuro/Psychometric_Test_Results_01.csv')
psychometric_02 = pd.read_csv('/Users/urmebose/Desktop/People/Nuro/Psychometric_Test_Results_02.csv')
psychometric_03 = pd.read_csv('/Users/urmebose/Desktop/People/Nuro/Psychometric_Test_Results_03.csv')

# Clean and prepare the psychometric data
psychometric_01.columns = ['Type', 'Test', 'Question', 'Answer', 'Time(s)', 'Question Start Time', 'Question Answer Time']
psychometric_02.columns = ['Type', 'Test', 'Question', 'Answer', 'Time(s)', 'Question Start Time', 'Question Answer Time']
psychometric_03.columns = ['Type', 'Test', 'Question', 'Answer', 'Time(s)', 'Question Start Time', 'Question Answer Time']

# Convert 'datetime' columns to datetime format and ensure they are tz-naive
hr_baseline['datetime'] = pd.to_datetime(hr_baseline['datetime'], errors='coerce').dt.tz_localize(None)
hr_01['datetime'] = pd.to_datetime(hr_01['datetime'], errors='coerce').dt.tz_localize(None)
hr_02['datetime'] = pd.to_datetime(hr_02['datetime'], errors='coerce').dt.tz_localize(None)
hr_03['datetime'] = pd.to_datetime(hr_03['datetime'], errors='coerce').dt.tz_localize(None)
psychometric_01['Question Start Time'] = pd.to_datetime(psychometric_01['Question Start Time'], errors='coerce').dt.tz_localize(None)
psychometric_02['Question Start Time'] = pd.to_datetime(psychometric_02['Question Start Time'], errors='coerce').dt.tz_localize(None)
psychometric_03['Question Start Time'] = pd.to_datetime(psychometric_03['Question Start Time'], errors='coerce').dt.tz_localize(None)
psychometric_01['Question Answer Time'] = pd.to_datetime(psychometric_01['Question Answer Time'], errors='coerce').dt.tz_localize(None)
psychometric_02['Question Answer Time'] = pd.to_datetime(psychometric_02['Question Answer Time'], errors='coerce').dt.tz_localize(None)
psychometric_03['Question Answer Time'] = pd.to_datetime(psychometric_03['Question Answer Time'], errors='coerce').dt.tz_localize(None)

# Remove rows with NaT values in datetime columns
hr_baseline = hr_baseline.dropna(subset=['datetime'])
hr_01 = hr_01.dropna(subset=['datetime'])
hr_02 = hr_02.dropna(subset=['datetime'])
hr_03 = hr_03.dropna(subset=['datetime'])
psychometric_01 = psychometric_01.dropna(subset=['Question Start Time'])
psychometric_02 = psychometric_02.dropna(subset=['Question Start Time'])
psychometric_03 = psychometric_03.dropna(subset=['Question Start Time'])

# Filter for HADS questions
questions_hads_01 = psychometric_01[psychometric_01['Type'] == 'HADS'].copy()
questions_hads_02 = psychometric_02[psychometric_02['Type'] == 'HADS'].copy()
questions_hads_03 = psychometric_03[psychometric_03['Type'] == 'HADS'].copy()

def get_average_hr_and_times(questions, hr_data):
    # Merge the psychometric data with heart rate data based on the closest timestamp
    questions.loc[:, 'closest_hr'] = questions['Question Start Time'].apply(
        lambda x: hr_data.iloc[(hr_data['datetime'] - x).abs().argsort()[:1]]['heart_rate'].values[0])
    # Calculate the average heart rate
    average_hr = questions['closest_hr'].mean()
    # Get the start and end times of answering
    start_time = questions['Question Start Time'].min().strftime('%H:%M:%S')
    end_time = questions['Question Answer Time'].max().strftime('%H:%M:%S')
    return average_hr, start_time, end_time

# Calculate average heart rate and times for baseline
baseline_avg_hr = hr_baseline['heart_rate'].mean()

# Calculate average heart rate and times for HADS during test sessions 01, 02, and 03
average_hr_01, start_time_01, end_time_01 = get_average_hr_and_times(questions_hads_01, hr_01)
average_hr_02, start_time_02, end_time_02 = get_average_hr_and_times(questions_hads_02, hr_02)
average_hr_03, start_time_03, end_time_03 = get_average_hr_and_times(questions_hads_03, hr_03)

# Determine if the HR indicates anxiety based on the baseline HR and general threshold
anxiety_individual_01 = average_hr_01 > baseline_avg_hr
anxiety_general_01 = average_hr_01 > 100
anxiety_individual_02 = average_hr_02 > baseline_avg_hr
anxiety_general_02 = average_hr_02 > 100
anxiety_individual_03 = average_hr_03 > baseline_avg_hr
anxiety_general_03 = average_hr_03 > 100

# Create a DataFrame to display the results in a table
results = pd.DataFrame({
    'Test': ['Test 01', 'Test 02', 'Test 03'],
    'Start Time': [start_time_01, start_time_02, start_time_03],
    'End Time': [end_time_01, end_time_02, end_time_03],
    'Average HR (BPM)': [average_hr_01, average_hr_02, average_hr_03],
    'Anxiety (Individual)': ['Yes' if anxiety_individual_01 else 'No', 'Yes' if anxiety_individual_02 else 'No', 'Yes' if anxiety_individual_03 else 'No'],
    'Anxiety (General >100 BPM)': ['Yes' if anxiety_general_01 else 'No', 'Yes' if anxiety_general_02 else 'No', 'Yes' if anxiety_general_03 else 'No']
})

# Display the results more clearly
print(f"Baseline Average HR: {baseline_avg_hr:.2f} BPM\n")

for index, row in results.iterrows():
    print(f"{row['Test']}:")
    print(f"  Start time for HADS: {row['Start Time']}")
    print(f"  End time for HADS: {row['End Time']}")
    print(f"  Average HR during HADS: {row['Average HR (BPM)']:.2f} BPM")
    print(f"  Anxiety (Individual Threshold): {row['Anxiety (Individual)']}")
    print(f"  Anxiety (General Threshold >100 BPM): {row['Anxiety (General >100 BPM)']}\n")

Baseline Average HR: 63.43 BPM

Test 01:
  Start time for HADS: 11:30:18
  End time for HADS: 11:32:10
  Average HR during HADS: 64.50 BPM
  Anxiety (Individual Threshold): Yes
  Anxiety (General Threshold >100 BPM): No

Test 02:
  Start time for HADS: 09:41:08
  End time for HADS: 09:42:25
  Average HR during HADS: 65.07 BPM
  Anxiety (Individual Threshold): Yes
  Anxiety (General Threshold >100 BPM): No

Test 03:
  Start time for HADS: 12:48:35
  End time for HADS: 12:49:38
  Average HR during HADS: 84.86 BPM
  Anxiety (Individual Threshold): Yes
  Anxiety (General Threshold >100 BPM): No



In [42]:
import pandas as pd

# Load HR data for baseline and all three seasons
hr_baseline = pd.read_csv('/Users/urmebose/Desktop/People/Nuro/hr.csv')
hr_01 = pd.read_csv('/Users/urmebose/Desktop/People/Nuro/hr_01.csv')
hr_02 = pd.read_csv('/Users/urmebose/Desktop/People/Nuro/hr_02.csv')
hr_03 = pd.read_csv('/Users/urmebose/Desktop/People/Nuro/hr_03.csv')

# Load psychometric test data for all three seasons
psychometric_01 = pd.read_csv('/Users/urmebose/Desktop/People/Nuro/Psychometric_Test_Results_01.csv')
psychometric_02 = pd.read_csv('/Users/urmebose/Desktop/People/Nuro/Psychometric_Test_Results_02.csv')
psychometric_03 = pd.read_csv('/Users/urmebose/Desktop/People/Nuro/Psychometric_Test_Results_03.csv')

# Clean and prepare the psychometric data
psychometric_01.columns = ['Type', 'Test', 'Question', 'Answer', 'Time(s)', 'Question Start Time', 'Question Answer Time']
psychometric_02.columns = ['Type', 'Test', 'Question', 'Answer', 'Time(s)', 'Question Start Time', 'Question Answer Time']
psychometric_03.columns = ['Type', 'Test', 'Question', 'Answer', 'Time(s)', 'Question Start Time', 'Question Answer Time']

# Convert 'datetime' columns to datetime format and ensure they are tz-naive
hr_baseline['datetime'] = pd.to_datetime(hr_baseline['datetime'], errors='coerce').dt.tz_localize(None)
hr_01['datetime'] = pd.to_datetime(hr_01['datetime'], errors='coerce').dt.tz_localize(None)
hr_02['datetime'] = pd.to_datetime(hr_02['datetime'], errors='coerce').dt.tz_localize(None)
hr_03['datetime'] = pd.to_datetime(hr_03['datetime'], errors='coerce').dt.tz_localize(None)
psychometric_01['Question Start Time'] = pd.to_datetime(psychometric_01['Question Start Time'], errors='coerce').dt.tz_localize(None)
psychometric_02['Question Start Time'] = pd.to_datetime(psychometric_02['Question Start Time'], errors='coerce').dt.tz_localize(None)
psychometric_03['Question Start Time'] = pd.to_datetime(psychometric_03['Question Start Time'], errors='coerce').dt.tz_localize(None)
psychometric_01['Question Answer Time'] = pd.to_datetime(psychometric_01['Question Answer Time'], errors='coerce').dt.tz_localize(None)
psychometric_02['Question Answer Time'] = pd.to_datetime(psychometric_02['Question Answer Time'], errors='coerce').dt.tz_localize(None)
psychometric_03['Question Answer Time'] = pd.to_datetime(psychometric_03['Question Answer Time'], errors='coerce').dt.tz_localize(None)

# Remove rows with NaT values in datetime columns
hr_baseline = hr_baseline.dropna(subset=['datetime'])
hr_01 = hr_01.dropna(subset=['datetime'])
hr_02 = hr_02.dropna(subset=['datetime'])
hr_03 = hr_03.dropna(subset=['datetime'])
psychometric_01 = psychometric_01.dropna(subset=['Question Start Time'])
psychometric_02 = psychometric_02.dropna(subset=['Question Start Time'])
psychometric_03 = psychometric_03.dropna(subset=['Question Start Time'])

# Filter for STAI-S questions
questions_stai_s_01 = psychometric_01[psychometric_01['Type'] == 'STAI-S'].copy()
questions_stai_s_02 = psychometric_02[psychometric_02['Type'] == 'STAI-S'].copy()
questions_stai_s_03 = psychometric_03[psychometric_03['Type'] == 'STAI-S'].copy()

def get_average_hr_and_times(questions, hr_data):
    # Merge the psychometric data with heart rate data based on the closest timestamp
    questions.loc[:, 'closest_hr'] = questions['Question Start Time'].apply(
        lambda x: hr_data.iloc[(hr_data['datetime'] - x).abs().argsort()[:1]]['heart_rate'].values[0])
    # Calculate the average heart rate
    average_hr = questions['closest_hr'].mean()
    # Get the start and end times of answering
    start_time = questions['Question Start Time'].min().strftime('%H:%M:%S')
    end_time = questions['Question Answer Time'].max().strftime('%H:%M:%S')
    return average_hr, start_time, end_time

# Calculate average heart rate and times for baseline
baseline_avg_hr = hr_baseline['heart_rate'].mean()

# Calculate average heart rate and times for STAI-S during test sessions 01, 02, and 03
average_hr_stai_s_01, start_time_stai_s_01, end_time_stai_s_01 = get_average_hr_and_times(questions_stai_s_01, hr_01)
average_hr_stai_s_02, start_time_stai_s_02, end_time_stai_s_02 = get_average_hr_and_times(questions_stai_s_02, hr_02)
average_hr_stai_s_03, start_time_stai_s_03, end_time_stai_s_03 = get_average_hr_and_times(questions_stai_s_03, hr_03)

# Determine if the HR indicates anxiety based on the baseline HR and general threshold
anxiety_individual_stai_s_01 = average_hr_stai_s_01 > baseline_avg_hr
anxiety_general_stai_s_01 = average_hr_stai_s_01 > 100
anxiety_individual_stai_s_02 = average_hr_stai_s_02 > baseline_avg_hr
anxiety_general_stai_s_02 = average_hr_stai_s_02 > 100
anxiety_individual_stai_s_03 = average_hr_stai_s_03 > baseline_avg_hr
anxiety_general_stai_s_03 = average_hr_stai_s_03 > 100

# Create a DataFrame to display the results in a table
results_stai_s = pd.DataFrame({
    'Test': ['Test 01', 'Test 02', 'Test 03'],
    'Start Time': [start_time_stai_s_01, start_time_stai_s_02, start_time_stai_s_03],
    'End Time': [end_time_stai_s_01, end_time_stai_s_02, end_time_stai_s_03],
    'Average HR (BPM)': [average_hr_stai_s_01, average_hr_stai_s_02, average_hr_stai_s_03],
    'Anxiety (Individual)': ['Yes' if anxiety_individual_stai_s_01 else 'No', 'Yes' if anxiety_individual_stai_s_02 else 'No', 'Yes' if anxiety_individual_stai_s_03 else 'No'],
    'Anxiety (General >100 BPM)': ['Yes' if anxiety_general_stai_s_01 else 'No', 'Yes' if anxiety_general_stai_s_02 else 'No', 'Yes' if anxiety_general_stai_s_03 else 'No']
})

# Display the results more clearly
print(f"Baseline Average HR: {baseline_avg_hr:.2f} BPM\n")

for index, row in results_stai_s.iterrows():
    print(f"{row['Test']}:")
    print(f"  Start time for STAI-S: {row['Start Time']}")
    print(f"  End time for STAI-S: {row['End Time']}")
    print(f"  Average HR during STAI-S: {row['Average HR (BPM)']:.2f} BPM")
    print(f"  Anxiety (Individual Threshold): {row['Anxiety (Individual)']}")
    print(f"  Anxiety (General Threshold >100 BPM): {row['Anxiety (General >100 BPM)']}\n")


Baseline Average HR: 63.43 BPM

Test 01:
  Start time for STAI-S: 11:32:10
  End time for STAI-S: 11:34:04
  Average HR during STAI-S: 63.30 BPM
  Anxiety (Individual Threshold): No
  Anxiety (General Threshold >100 BPM): No

Test 02:
  Start time for STAI-S: 09:42:25
  End time for STAI-S: 09:43:40
  Average HR during STAI-S: 64.75 BPM
  Anxiety (Individual Threshold): Yes
  Anxiety (General Threshold >100 BPM): No

Test 03:
  Start time for STAI-S: 12:49:38
  End time for STAI-S: 12:50:32
  Average HR during STAI-S: 86.10 BPM
  Anxiety (Individual Threshold): Yes
  Anxiety (General Threshold >100 BPM): No



In [46]:
import pandas as pd

# Load HR data for baseline and all three seasons
hr_baseline = pd.read_csv('/Users/urmebose/Desktop/People/Nuro/hr.csv')
hr_01 = pd.read_csv('/Users/urmebose/Desktop/People/Nuro/hr_01.csv')
hr_02 = pd.read_csv('/Users/urmebose/Desktop/People/Nuro/hr_02.csv')
hr_03 = pd.read_csv('/Users/urmebose/Desktop/People/Nuro/hr_03.csv')

# Load psychometric test data for all three seasons
psychometric_01 = pd.read_csv('/Users/urmebose/Desktop/People/Nuro/Psychometric_Test_Results_01.csv')
psychometric_02 = pd.read_csv('/Users/urmebose/Desktop/People/Nuro/Psychometric_Test_Results_02.csv')
psychometric_03 = pd.read_csv('/Users/urmebose/Desktop/People/Nuro/Psychometric_Test_Results_03.csv')

# Clean and prepare the psychometric data
psychometric_01.columns = ['Type', 'Test', 'Question', 'Answer', 'Time(s)', 'Question Start Time', 'Question Answer Time']
psychometric_02.columns = ['Type', 'Test', 'Question', 'Answer', 'Time(s)', 'Question Start Time', 'Question Answer Time']
psychometric_03.columns = ['Type', 'Test', 'Question', 'Answer', 'Time(s)', 'Question Start Time', 'Question Answer Time']

# Convert 'datetime' columns to datetime format and ensure they are tz-naive
hr_baseline['datetime'] = pd.to_datetime(hr_baseline['datetime'], errors='coerce').dt.tz_localize(None)
hr_01['datetime'] = pd.to_datetime(hr_01['datetime'], errors='coerce').dt.tz_localize(None)
hr_02['datetime'] = pd.to_datetime(hr_02['datetime'], errors='coerce').dt.tz_localize(None)
hr_03['datetime'] = pd.to_datetime(hr_03['datetime'], errors='coerce').dt.tz_localize(None)
psychometric_01['Question Start Time'] = pd.to_datetime(psychometric_01['Question Start Time'], errors='coerce').dt.tz_localize(None)
psychometric_02['Question Start Time'] = pd.to_datetime(psychometric_02['Question Start Time'], errors='coerce').dt.tz_localize(None)
psychometric_03['Question Start Time'] = pd.to_datetime(psychometric_03['Question Start Time'], errors='coerce').dt.tz_localize(None)
psychometric_01['Question Answer Time'] = pd.to_datetime(psychometric_01['Question Answer Time'], errors='coerce').dt.tz_localize(None)
psychometric_02['Question Answer Time'] = pd.to_datetime(psychometric_02['Question Answer Time'], errors='coerce').dt.tz_localize(None)
psychometric_03['Question Answer Time'] = pd.to_datetime(psychometric_03['Question Answer Time'], errors='coerce').dt.tz_localize(None)

# Remove rows with NaT values in datetime columns
hr_baseline = hr_baseline.dropna(subset=['datetime'])
hr_01 = hr_01.dropna(subset=['datetime'])
hr_02 = hr_02.dropna(subset=['datetime'])
hr_03 = hr_03.dropna(subset=['datetime'])
psychometric_01 = psychometric_01.dropna(subset=['Question Start Time'])
psychometric_02 = psychometric_02.dropna(subset=['Question Start Time'])
psychometric_03 = psychometric_03.dropna(subset=['Question Start Time'])

# Filter for STAI-T questions
questions_stai_t_01 = psychometric_01[psychometric_01['Type'] == 'STAI-T'].copy()
questions_stai_t_02 = psychometric_02[psychometric_02['Type'] == 'STAI-T'].copy()
questions_stai_t_03 = psychometric_03[psychometric_03['Type'] == 'STAI-T'].copy()

def get_average_hr_and_times(questions, hr_data):
    # Merge the psychometric data with heart rate data based on the closest timestamp
    questions.loc[:, 'closest_hr'] = questions['Question Start Time'].apply(
        lambda x: hr_data.iloc[(hr_data['datetime'] - x).abs().argsort()[:1]]['heart_rate'].values[0])
    # Calculate the average heart rate
    average_hr = questions['closest_hr'].mean()
    # Get the start and end times of answering
    start_time = questions['Question Start Time'].min().strftime('%H:%M:%S')
    end_time = questions['Question Answer Time'].max().strftime('%H:%M:%S')
    return average_hr, start_time, end_time

# Calculate average heart rate and times for baseline
baseline_avg_hr = hr_baseline['heart_rate'].mean()

# Calculate average heart rate and times for STAI-T during test sessions 01, 02, and 03
average_hr_stai_t_01, start_time_stai_t_01, end_time_stai_t_01 = get_average_hr_and_times(questions_stai_t_01, hr_01)
average_hr_stai_t_02, start_time_stai_t_02, end_time_stai_t_02 = get_average_hr_and_times(questions_stai_t_02, hr_02)
average_hr_stai_t_03, start_time_stai_t_03, end_time_stai_t_03 = get_average_hr_and_times(questions_stai_t_03, hr_03)

# Determine if the HR indicates anxiety based on the baseline HR and general threshold
anxiety_individual_stai_t_01 = average_hr_stai_t_01 > baseline_avg_hr
anxiety_general_stai_t_01 = average_hr_stai_t_01 > 100
anxiety_individual_stai_t_02 = average_hr_stai_t_02 > baseline_avg_hr
anxiety_general_stai_t_02 = average_hr_stai_t_02 > 100
anxiety_individual_stai_t_03 = average_hr_stai_t_03 > baseline_avg_hr
anxiety_general_stai_t_03 = average_hr_stai_t_03 > 100

# Create a DataFrame to display the results in a table
results_stai_t = pd.DataFrame({
    'Test': ['Test 01', 'Test 02', 'Test 03'],
    'Start Time': [start_time_stai_t_01, start_time_stai_t_02, start_time_stai_t_03],
    'End Time': [end_time_stai_t_01, end_time_stai_t_02, end_time_stai_t_03],
    'Average HR (BPM)': [average_hr_stai_t_01, average_hr_stai_t_02, average_hr_stai_t_03],
    'Anxiety (Individual)': ['Yes' if anxiety_individual_stai_t_01 else 'No', 'Yes' if anxiety_individual_stai_t_02 else 'No', 'Yes' if anxiety_individual_stai_t_03 else 'No'],
    'Anxiety (General >100 BPM)': ['Yes' if anxiety_general_stai_t_01 else 'No', 'Yes' if anxiety_general_stai_t_02 else 'No', 'Yes' if anxiety_general_stai_t_03 else 'No']
})

# Display the results more clearly
print(f"Baseline Average HR: {baseline_avg_hr:.2f} BPM\n")

for index, row in results_stai_t.iterrows():
    print(f"{row['Test']}:")
    print(f"  Start time for STAI-T: {row['Start Time']}")
    print(f"  End time for STAI-T: {row['End Time']}")
    print(f"  Average HR during STAI-T: {row['Average HR (BPM)']:.2f} BPM")
    print(f"  Anxiety (Individual Threshold): {row['Anxiety (Individual)']}")
    print(f"  Anxiety (General Threshold >100 BPM): {row['Anxiety (General >100 BPM)']}\n")

Baseline Average HR: 63.43 BPM

Test 01:
  Start time for STAI-T: 11:34:04
  End time for STAI-T: 11:35:30
  Average HR during STAI-T: 64.75 BPM
  Anxiety (Individual Threshold): Yes
  Anxiety (General Threshold >100 BPM): No

Test 02:
  Start time for STAI-T: 09:43:40
  End time for STAI-T: 09:45:00
  Average HR during STAI-T: 64.10 BPM
  Anxiety (Individual Threshold): Yes
  Anxiety (General Threshold >100 BPM): No

Test 03:
  Start time for STAI-T: 12:50:32
  End time for STAI-T: 12:51:30
  Average HR during STAI-T: 87.05 BPM
  Anxiety (Individual Threshold): Yes
  Anxiety (General Threshold >100 BPM): No



In [54]:
import pandas as pd

# Load HR data for baseline and all three seasons
hr_baseline = pd.read_csv('/Users/urmebose/Desktop/People/Nuro/hr.csv')
hr_01 = pd.read_csv('/Users/urmebose/Desktop/People/Nuro/hr_01.csv')
hr_02 = pd.read_csv('/Users/urmebose/Desktop/People/Nuro/hr_02.csv')
hr_03 = pd.read_csv('/Users/urmebose/Desktop/People/Nuro/hr_03.csv')

# Load psychometric test data for all three seasons
psychometric_01 = pd.read_csv('/Users/urmebose/Desktop/People/Nuro/Psychometric_Test_Results_01.csv')
psychometric_02 = pd.read_csv('/Users/urmebose/Desktop/People/Nuro/Psychometric_Test_Results_02.csv')
psychometric_03 = pd.read_csv('/Users/urmebose/Desktop/People/Nuro/Psychometric_Test_Results_03.csv')

# Clean and prepare the psychometric data
psychometric_01.columns = ['Type', 'Test', 'Question', 'Answer', 'Time(s)', 'Question Start Time', 'Question Answer Time']
psychometric_02.columns = ['Type', 'Test', 'Question', 'Answer', 'Time(s)', 'Question Start Time', 'Question Answer Time']
psychometric_03.columns = ['Type', 'Test', 'Question', 'Answer', 'Time(s)', 'Question Start Time', 'Question Answer Time']

# Convert 'datetime' columns to datetime format and ensure they are tz-naive
hr_baseline['datetime'] = pd.to_datetime(hr_baseline['datetime'], errors='coerce').dt.tz_localize(None)
hr_01['datetime'] = pd.to_datetime(hr_01['datetime'], errors='coerce').dt.tz_localize(None)
hr_02['datetime'] = pd.to_datetime(hr_02['datetime'], errors='coerce').dt.tz_localize(None)
hr_03['datetime'] = pd.to_datetime(hr_03['datetime'], errors='coerce').dt.tz_localize(None)
psychometric_01['Question Start Time'] = pd.to_datetime(psychometric_01['Question Start Time'], errors='coerce').dt.tz_localize(None)
psychometric_02['Question Start Time'] = pd.to_datetime(psychometric_02['Question Start Time'], errors='coerce').dt.tz_localize(None)
psychometric_03['Question Start Time'] = pd.to_datetime(psychometric_03['Question Start Time'], errors='coerce').dt.tz_localize(None)
psychometric_01['Question Answer Time'] = pd.to_datetime(psychometric_01['Question Answer Time'], errors='coerce').dt.tz_localize(None)
psychometric_02['Question Answer Time'] = pd.to_datetime(psychometric_02['Question Answer Time'], errors='coerce').dt.tz_localize(None)
psychometric_03['Question Answer Time'] = pd.to_datetime(psychometric_03['Question Answer Time'], errors='coerce').dt.tz_localize(None)

# Remove rows with NaT values in datetime columns
hr_baseline = hr_baseline.dropna(subset=['datetime'])
hr_01 = hr_01.dropna(subset=['datetime'])
hr_02 = hr_02.dropna(subset=['datetime'])
hr_03 = hr_03.dropna(subset=['datetime'])
psychometric_01 = psychometric_01.dropna(subset=['Question Start Time'])
psychometric_02 = psychometric_02.dropna(subset=['Question Start Time'])
psychometric_03 = psychometric_03.dropna(subset=['Question Start Time'])

# Filter for BFI questions
questions_bfi_01 = psychometric_01[psychometric_01['Type'] == 'BFI'].copy()
questions_bfi_02 = psychometric_02[psychometric_02['Type'] == 'BFI'].copy()
questions_bfi_03 = psychometric_03[psychometric_03['Type'] == 'BFI'].copy()

def get_average_hr_and_times(questions, hr_data):
    # Merge the psychometric data with heart rate data based on the closest timestamp
    questions.loc[:, 'closest_hr'] = questions['Question Start Time'].apply(
        lambda x: hr_data.iloc[(hr_data['datetime'] - x).abs().argsort()[:1]]['heart_rate'].values[0])
    # Calculate the average heart rate
    average_hr = questions['closest_hr'].mean()
    # Get the start and end times of answering
    start_time = questions['Question Start Time'].min().strftime('%H:%M:%S')
    end_time = questions['Question Answer Time'].max().strftime('%H:%M:%S')
    return average_hr, start_time, end_time

# Calculate average heart rate and times for baseline
baseline_avg_hr = hr_baseline['heart_rate'].mean()

# Calculate average heart rate and times for BFI during test sessions 01, 02, and 03
average_hr_bfi_01, start_time_bfi_01, end_time_bfi_01 = get_average_hr_and_times(questions_bfi_01, hr_01)
average_hr_bfi_02, start_time_bfi_02, end_time_bfi_02 = get_average_hr_and_times(questions_bfi_02, hr_02)
average_hr_bfi_03, start_time_bfi_03, end_time_bfi_03 = get_average_hr_and_times(questions_bfi_03, hr_03)

# Determine if the HR indicates anxiety based on the baseline HR and general threshold
anxiety_individual_bfi_01 = average_hr_bfi_01 > baseline_avg_hr
anxiety_general_bfi_01 = average_hr_bfi_01 > 100
anxiety_individual_bfi_02 = average_hr_bfi_02 > baseline_avg_hr
anxiety_general_bfi_02 = average_hr_bfi_02 > 100
anxiety_individual_bfi_03 = average_hr_bfi_03 > baseline_avg_hr
anxiety_general_bfi_03 = average_hr_bfi_03 > 100

# Create a DataFrame to display the results in a table
results_bfi = pd.DataFrame({
    'Test': ['Test 01', 'Test 02', 'Test 03'],
    'Start Time': [start_time_bfi_01, start_time_bfi_02, start_time_bfi_03],
    'End Time': [end_time_bfi_01, end_time_bfi_02, end_time_bfi_03],
    'Average HR (BPM)': [average_hr_bfi_01, average_hr_bfi_02, average_hr_bfi_03],
    'Anxiety (Individual)': ['Yes' if anxiety_individual_bfi_01 else 'No', 'Yes' if anxiety_individual_bfi_02 else 'No', 'Yes' if anxiety_individual_bfi_03 else 'No'],
    'Anxiety (General >100 BPM)': ['Yes' if anxiety_general_bfi_01 else 'No', 'Yes' if anxiety_general_bfi_02 else 'No', 'Yes' if anxiety_general_bfi_03 else 'No']
})

# Display the results more clearly
print(f"Baseline Average HR: {baseline_avg_hr:.2f} BPM\n")

for index, row in results_bfi.iterrows():
    print(f"{row['Test']}:")
    print(f"  Start time for BFI: {row['Start Time']}")
    print(f"  End time for BFI: {row['End Time']}")
    print(f"  Average HR during BFI: {row['Average HR (BPM)']:.2f} BPM")
    print(f"  Anxiety (Individual Threshold): {row['Anxiety (Individual)']}")
    print(f"  Anxiety (General Threshold >100 BPM): {row['Anxiety (General >100 BPM)']}\n")

Baseline Average HR: 63.43 BPM

Test 01:
  Start time for BFI: 11:35:30
  End time for BFI: 11:36:06
  Average HR during BFI: 67.20 BPM
  Anxiety (Individual Threshold): Yes
  Anxiety (General Threshold >100 BPM): No

Test 02:
  Start time for BFI: 09:45:00
  End time for BFI: 09:45:39
  Average HR during BFI: 64.60 BPM
  Anxiety (Individual Threshold): Yes
  Anxiety (General Threshold >100 BPM): No

Test 03:
  Start time for BFI: 12:51:30
  End time for BFI: 12:52:06
  Average HR during BFI: 86.50 BPM
  Anxiety (Individual Threshold): Yes
  Anxiety (General Threshold >100 BPM): No



In [58]:
import pandas as pd

# Load HR data for baseline and all three seasons
hr_baseline = pd.read_csv('/Users/urmebose/Desktop/People/Nuro/hr.csv')
hr_01 = pd.read_csv('/Users/urmebose/Desktop/People/Nuro/hr_01.csv')
hr_02 = pd.read_csv('/Users/urmebose/Desktop/People/Nuro/hr_02.csv')
hr_03 = pd.read_csv('/Users/urmebose/Desktop/People/Nuro/hr_03.csv')

# Load psychometric test data for all three seasons
psychometric_01 = pd.read_csv('/Users/urmebose/Desktop/People/Nuro/Psychometric_Test_Results_01.csv')
psychometric_02 = pd.read_csv('/Users/urmebose/Desktop/People/Nuro/Psychometric_Test_Results_02.csv')
psychometric_03 = pd.read_csv('/Users/urmebose/Desktop/People/Nuro/Psychometric_Test_Results_03.csv')

# Clean and prepare the psychometric data
psychometric_01.columns = ['Type', 'Test', 'Question', 'Answer', 'Time(s)', 'Question Start Time', 'Question Answer Time']
psychometric_02.columns = ['Type', 'Test', 'Question', 'Answer', 'Time(s)', 'Question Start Time', 'Question Answer Time']
psychometric_03.columns = ['Type', 'Test', 'Question', 'Answer', 'Time(s)', 'Question Start Time', 'Question Answer Time']

# Convert 'datetime' columns to datetime format and ensure they are tz-naive
hr_baseline['datetime'] = pd.to_datetime(hr_baseline['datetime'], errors='coerce').dt.tz_localize(None)
hr_01['datetime'] = pd.to_datetime(hr_01['datetime'], errors='coerce').dt.tz_localize(None)
hr_02['datetime'] = pd.to_datetime(hr_02['datetime'], errors='coerce').dt.tz_localize(None)
hr_03['datetime'] = pd.to_datetime(hr_03['datetime'], errors='coerce').dt.tz_localize(None)
psychometric_01['Question Start Time'] = pd.to_datetime(psychometric_01['Question Start Time'], errors='coerce').dt.tz_localize(None)
psychometric_02['Question Start Time'] = pd.to_datetime(psychometric_02['Question Start Time'], errors='coerce').dt.tz_localize(None)
psychometric_03['Question Start Time'] = pd.to_datetime(psychometric_03['Question Start Time'], errors='coerce').dt.tz_localize(None)
psychometric_01['Question Answer Time'] = pd.to_datetime(psychometric_01['Question Answer Time'], errors='coerce').dt.tz_localize(None)
psychometric_02['Question Answer Time'] = pd.to_datetime(psychometric_02['Question Answer Time'], errors='coerce').dt.tz_localize(None)
psychometric_03['Question Answer Time'] = pd.to_datetime(psychometric_03['Question Answer Time'], errors='coerce').dt.tz_localize(None)

# Remove rows with NaT values in datetime columns
hr_baseline = hr_baseline.dropna(subset=['datetime'])
hr_01 = hr_01.dropna(subset=['datetime'])
hr_02 = hr_02.dropna(subset=['datetime'])
hr_03 = hr_03.dropna(subset=['datetime'])
psychometric_01 = psychometric_01.dropna(subset=['Question Start Time'])
psychometric_02 = psychometric_02.dropna(subset=['Question Start Time'])
psychometric_03 = psychometric_03.dropna(subset=['Question Start Time'])


# Filter for BFI questions
questions_bfi_01 = psychometric_01[psychometric_01['Type'] == 'BFI'].copy()
questions_bfi_02 = psychometric_02[psychometric_02['Type'] == 'BFI'].copy()
questions_bfi_03 = psychometric_03[psychometric_03['Type'] == 'BFI'].copy()

def get_average_hr_and_times(questions, hr_data):
    # Merge the psychometric data with heart rate data based on the closest timestamp
    questions.loc[:, 'closest_hr'] = questions['Question Start Time'].apply(
        lambda x: hr_data.iloc[(hr_data['datetime'] - x).abs().argsort()[:1]]['heart_rate'].values[0])
    # Calculate the average heart rate
    average_hr = questions['closest_hr'].mean()
    # Get the start and end times of answering
    start_time = questions['Question Start Time'].min().strftime('%H:%M:%S')
    end_time = questions['Question Answer Time'].max().strftime('%H:%M:%S')
    return average_hr, start_time, end_time

# Calculate average heart rate and times for baseline
baseline_avg_hr = hr_baseline['heart_rate'].mean()

# Calculate average heart rate and times for BFI during test sessions 01, 02, and 03
average_hr_bfi_01, start_time_bfi_01, end_time_bfi_01 = get_average_hr_and_times(questions_bfi_01, hr_01)
average_hr_bfi_02, start_time_bfi_02, end_time_bfi_02 = get_average_hr_and_times(questions_bfi_02, hr_02)
average_hr_bfi_03, start_time_bfi_03, end_time_bfi_03 = get_average_hr_and_times(questions_bfi_03, hr_03)

# Determine if the HR indicates anxiety based on the baseline HR and general threshold
anxiety_individual_bfi_01 = average_hr_bfi_01 > baseline_avg_hr
anxiety_general_bfi_01 = average_hr_bfi_01 > 100
anxiety_individual_bfi_02 = average_hr_bfi_02 > baseline_avg_hr
anxiety_general_bfi_02 = average_hr_bfi_02 > 100
anxiety_individual_bfi_03 = average_hr_bfi_03 > baseline_avg_hr
anxiety_general_bfi_03 = average_hr_bfi_03 > 100

# Create a DataFrame to display the results in a table
results_bfi = pd.DataFrame({
    'Test': ['Test 01', 'Test 02', 'Test 03'],
    'Start Time': [start_time_bfi_01, start_time_bfi_02, start_time_bfi_03],
    'End Time': [end_time_bfi_01, end_time_bfi_02, end_time_bfi_03],
    'Average HR (BPM)': [average_hr_bfi_01, average_hr_bfi_02, average_hr_bfi_03],
    'Anxiety (Individual)': ['Yes' if anxiety_individual_bfi_01 else 'No', 'Yes' if anxiety_individual_bfi_02 else 'No', 'Yes' if anxiety_individual_bfi_03 else 'No'],
    'Anxiety (General >100 BPM)': ['Yes' if anxiety_general_bfi_01 else 'No', 'Yes' if anxiety_general_bfi_02 else 'No', 'Yes' if anxiety_general_bfi_03 else 'No']
})

# Display the results more clearly
print(f"Baseline Average HR: {baseline_avg_hr:.2f} BPM\n")

for index, row in results_bfi.iterrows():
    print(f"{row['Test']}:")
    print(f"  Start time for BFI: {row['Start Time']}")
    print(f"  End time for BFI: {row['End Time']}")
    print(f"  Average HR during BFI: {row['Average HR (BPM)']:.2f} BPM")
    print(f"  Anxiety (Individual Threshold): {row['Anxiety (Individual)']}")
    print(f"  Anxiety (General Threshold >100 BPM): {row['Anxiety (General >100 BPM)']}\n")

Baseline Average HR: 63.43 BPM

Test 01:
  Start time for BFI: 11:35:30
  End time for BFI: 11:36:06
  Average HR during BFI: 67.20 BPM
  Anxiety (Individual Threshold): Yes
  Anxiety (General Threshold >100 BPM): No

Test 02:
  Start time for BFI: 09:45:00
  End time for BFI: 09:45:39
  Average HR during BFI: 64.60 BPM
  Anxiety (Individual Threshold): Yes
  Anxiety (General Threshold >100 BPM): No

Test 03:
  Start time for BFI: 12:51:30
  End time for BFI: 12:52:06
  Average HR during BFI: 86.50 BPM
  Anxiety (Individual Threshold): Yes
  Anxiety (General Threshold >100 BPM): No



In [62]:
import pandas as pd

# Load HR data for baseline and all three seasons
hr_baseline = pd.read_csv('/Users/urmebose/Desktop/People/Nuro/hr.csv')
hr_01 = pd.read_csv('/Users/urmebose/Desktop/People/Nuro/hr_01.csv')
hr_02 = pd.read_csv('/Users/urmebose/Desktop/People/Nuro/hr_02.csv')
hr_03 = pd.read_csv('/Users/urmebose/Desktop/People/Nuro/hr_03.csv')

# Load psychometric test data for all three seasons
psychometric_01 = pd.read_csv('/Users/urmebose/Desktop/People/Nuro/Psychometric_Test_Results_01.csv')
psychometric_02 = pd.read_csv('/Users/urmebose/Desktop/People/Nuro/Psychometric_Test_Results_02.csv')
psychometric_03 = pd.read_csv('/Users/urmebose/Desktop/People/Nuro/Psychometric_Test_Results_03.csv')

# Clean and prepare the psychometric data
psychometric_01.columns = ['Type', 'Test', 'Question', 'Answer', 'Time(s)', 'Question Start Time', 'Question Answer Time']
psychometric_02.columns = ['Type', 'Test', 'Question', 'Answer', 'Time(s)', 'Question Start Time', 'Question Answer Time']
psychometric_03.columns = ['Type', 'Test', 'Question', 'Answer', 'Time(s)', 'Question Start Time', 'Question Answer Time']

# Convert 'datetime' columns to datetime format and ensure they are tz-naive
hr_baseline['datetime'] = pd.to_datetime(hr_baseline['datetime'], errors='coerce').dt.tz_localize(None)
hr_01['datetime'] = pd.to_datetime(hr_01['datetime'], errors='coerce').dt.tz_localize(None)
hr_02['datetime'] = pd.to_datetime(hr_02['datetime'], errors='coerce').dt.tz_localize(None)
hr_03['datetime'] = pd.to_datetime(hr_03['datetime'], errors='coerce').dt.tz_localize(None)
psychometric_01['Question Start Time'] = pd.to_datetime(psychometric_01['Question Start Time'], errors='coerce').dt.tz_localize(None)
psychometric_02['Question Start Time'] = pd.to_datetime(psychometric_02['Question Start Time'], errors='coerce').dt.tz_localize(None)
psychometric_03['Question Start Time'] = pd.to_datetime(psychometric_03['Question Start Time'], errors='coerce').dt.tz_localize(None)
psychometric_01['Question Answer Time'] = pd.to_datetime(psychometric_01['Question Answer Time'], errors='coerce').dt.tz_localize(None)
psychometric_02['Question Answer Time'] = pd.to_datetime(psychometric_02['Question Answer Time'], errors='coerce').dt.tz_localize(None)
psychometric_03['Question Answer Time'] = pd.to_datetime(psychometric_03['Question Answer Time'], errors='coerce').dt.tz_localize(None)

# Remove rows with NaT values in datetime columns
hr_baseline = hr_baseline.dropna(subset=['datetime'])
hr_01 = hr_01.dropna(subset=['datetime'])
hr_02 = hr_02.dropna(subset=['datetime'])
hr_03 = hr_03.dropna(subset(['datetime'])
psychometric_01 = psychometric_01.dropna(subset(['Question Start Time'])
psychometric_02 = psychometric_02.dropna(subset(['Question Start Time'])
psychometric_03 = psychometric_03.dropna(subset(['Question Start Time'])

# Filter for FQ questions
questions_fq_01 = psychometric_01[psychometric_01['Type'] == 'FQ'].copy()
questions_fq_02 = psychometric_02[psychometric_02['Type'] == 'FQ'].copy()
questions_fq_03 = psychometric_03[psychometric_03['Type'] == 'FQ'].copy()

def get_average_hr_and_times(questions, hr_data):
    # Merge the psychometric data with heart rate data based on the closest timestamp
    questions.loc[:, 'closest_hr'] = questions['Question Start Time'].apply(
        lambda x: hr_data.iloc[(hr_data['datetime'] - x).abs().argsort()[:1]]['heart_rate'].values[0])
    # Calculate the average heart rate
    average_hr = questions['closest_hr'].mean()
    # Get the start and end times of answering
    start_time = questions['Question Start Time'].min().strftime('%H:%M:%S')
    end_time = questions['Question Answer Time'].max().strftime('%H:%M:%S')
    return average_hr, start_time, end_time

# Calculate average heart rate and times for baseline
baseline_avg_hr = hr_baseline['heart_rate'].mean()

# Calculate average heart rate and times for FQ during test sessions 01, 02, and 03
average_hr_fq_01, start_time_fq_01, end_time_fq_01 = get_average_hr_and_times(questions_fq_01, hr_01)
average_hr_fq_02, start_time_fq_02, end_time_fq_02 = get_average_hr_and_times(questions_fq_02, hr_02)
average_hr_fq_03, start_time_fq_03, end_time_fq_03 = get_average_hr_and_times(questions_fq_03, hr_03)

# Determine if the HR indicates anxiety based on the baseline HR and general threshold
anxiety_individual_fq_01 = average_hr_fq_01 > baseline_avg_hr
anxiety_general_fq_01 = average_hr_fq_01 > 100
anxiety_individual_fq_02 = average_hr_fq_02 > baseline_avg_hr
anxiety_general_fq_02 = average_hr_fq_02 > 100
anxiety_individual_fq_03 = average_hr_fq_03 > baseline_avg_hr
anxiety_general_fq_03 = average_hr_fq_03 > 100

# Create a DataFrame to display the results in a table
results_fq = pd.DataFrame({
    'Test': ['Test 01', 'Test 02', 'Test 03'],
    'Start Time': [start_time_fq_01, start_time_fq_02, start_time_fq_03],
    'End Time': [end_time_fq_01, end_time_fq_02, end_time_fq_03],
    'Average HR (BPM)': [average_hr_fq_01, average_hr_fq_02, average_hr_fq_03],
    'Anxiety (Individual)': ['Yes' if anxiety_individual_fq_01 else 'No', 'Yes' if anxiety_individual_fq_02 else 'No', 'Yes' if anxiety_individual_fq_03 else 'No'],
    'Anxiety (General >100 BPM)': ['Yes' if anxiety_general_fq_01 else 'No', 'Yes' if anxiety_general_fq_02 else 'No', 'Yes' if anxiety_general_fq_03 else 'No']
})

# Display the results more clearly
print(f"Baseline Average HR: {baseline_avg_hr:.2f} BPM\n")

for index, row in results_fq.iterrows():
    print(f"{row['Test']}:")
    print(f"  Start time for FQ: {row['Start Time']}")
    print(f"  End time for FQ: {row['End Time']}")
    print(f"  Average HR during FQ: {row['Average HR (BPM)']:.2f} BPM")
    print(f"  Anxiety (Individual Threshold): {row['Anxiety (Individual)']}")
    print(f"  Anxiety (General Threshold >100 BPM): {row['Anxiety (General >100 BPM)']}\n")


Baseline Average HR: 63.43 BPM

Test 01:
  Start time for FQ: 11:36:06
  End time for FQ: 11:38:46
  Average HR during FQ: 64.08 BPM
  Anxiety (Individual Threshold): Yes
  Anxiety (General Threshold >100 BPM): No

Test 02:
  Start time for FQ: 09:45:39
  End time for FQ: 09:47:38
  Average HR during FQ: 64.92 BPM
  Anxiety (Individual Threshold): Yes
  Anxiety (General Threshold >100 BPM): No

Test 03:
  Start time for FQ: 12:52:06
  End time for FQ: 12:53:36
  Average HR during FQ: 87.08 BPM
  Anxiety (Individual Threshold): Yes
  Anxiety (General Threshold >100 BPM): No



In [64]:
import pandas as pd

# Load HR data for baseline and all three seasons
hr_baseline = pd.read_csv('/Users/urmebose/Desktop/People/Nuro/hr.csv')
hr_01 = pd.read_csv('/Users/urmebose/Desktop/People/Nuro/hr_01.csv')
hr_02 = pd.read_csv('/Users/urmebose/Desktop/People/Nuro/hr_02.csv')
hr_03 = pd.read_csv('/Users/urmebose/Desktop/People/Nuro/hr_03.csv')

# Load psychometric test data for all three seasons
psychometric_01 = pd.read_csv('/Users/urmebose/Desktop/People/Nuro/Psychometric_Test_Results_01.csv')
psychometric_02 = pd.read_csv('/Users/urmebose/Desktop/People/Nuro/Psychometric_Test_Results_02.csv')
psychometric_03 = pd.read_csv('/Users/urmebose/Desktop/People/Nuro/Psychometric_Test_Results_03.csv')

# Clean and prepare the psychometric data
columns = ['Type', 'Test', 'Question', 'Answer', 'Time(s)', 'Question Start Time', 'Question Answer Time']
for df in [psychometric_01, psychometric_02, psychometric_03]:
    df.columns = columns
    df['Question Start Time'] = pd.to_datetime(df['Question Start Time'], errors='coerce').dt.tz_localize(None)
    df['Question Answer Time'] = pd.to_datetime(df['Question Answer Time'], errors='coerce').dt.tz_localize(None)

# Process heart rate data
for df in [hr_baseline, hr_01, hr_02, hr_03]:
    df['datetime'] = pd.to_datetime(df['datetime'], errors='coerce').dt.tz_localize(None)

# Define function to calculate metrics
def get_average_hr_and_times(questions, hr_data):
    questions['closest_hr'] = questions['Question Start Time'].apply(
        lambda x: hr_data.iloc[(hr_data['datetime'] - x).abs().argsort()[:1]]['heart_rate'].values[0])
    average_hr = questions['closest_hr'].mean()
    start_time = questions['Question Start Time'].min().strftime('%H:%M:%S')
    end_time = questions['Question Answer Time'].max().strftime('%H:%M:%S')
    return average_hr, start_time, end_time

# Calculate and display results
baseline_avg_hr = hr_baseline['heart_rate'].mean()
test_types = ['HADS', 'STAI-S', 'STAI-T', 'BFI', 'FQ']
test_data = []
for test_type in test_types:
    for i in range(1, 4):
        psychometric = globals()[f'psychometric_0{i}']
        hr_data = globals()[f'hr_0{i}']
        questions = psychometric[psychometric['Type'] == test_type].copy()
        avg_hr, start_time, end_time = get_average_hr_and_times(questions, hr_data)
        individual_anxiety = 'Yes' if avg_hr > baseline_avg_hr else 'No'
        general_anxiety = 'Yes' if avg_hr > 100 else 'No'
        test_data.append((f'Test {i}', test_type, start_time, end_time, avg_hr, individual_anxiety, general_anxiety))

# Create a DataFrame to display the results in a table
results = pd.DataFrame(test_data, columns=['Test', 'Type', 'Start Time', 'End Time', 'Average HR (BPM)', 'Anxiety (Individual)', 'Anxiety (General >100 BPM)'])

# Display the results
print(f"Baseline Average HR: {baseline_avg_hr:.2f} BPM\n")
print(results.to_string(index=False))

Baseline Average HR: 63.43 BPM

  Test   Type Start Time End Time  Average HR (BPM) Anxiety (Individual) Anxiety (General >100 BPM)
Test 1   HADS   11:30:18 11:32:10         64.500000                  Yes                         No
Test 2   HADS   09:41:08 09:42:25         65.071429                  Yes                         No
Test 3   HADS   12:48:35 12:49:38         84.857143                  Yes                         No
Test 1 STAI-S   11:32:10 11:34:04         63.300000                   No                         No
Test 2 STAI-S   09:42:25 09:43:40         64.750000                  Yes                         No
Test 3 STAI-S   12:49:38 12:50:32         86.100000                  Yes                         No
Test 1 STAI-T   11:34:04 11:35:30         64.750000                  Yes                         No
Test 2 STAI-T   09:43:40 09:45:00         64.100000                  Yes                         No
Test 3 STAI-T   12:50:32 12:51:30         87.050000                 